# Chapter 10: 분포 변화 탐지 (Distribution Shifts)

이 노트북에서는 AI 에이전트 입력 데이터의 분포 변화를 탐지하는 방법을 학습합니다.

## 주요 내용
- KS (Kolmogorov-Smirnov) 검정
- KL (Kullback-Leibler) 발산
- PSI (Population Stability Index)
- 임베딩 기반 유사도 분석

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch10_distribution_shifts.ipynb)


In [ ]:
!pip install -q numpy scipy scikit-learn


In [ ]:
import numpy as np
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

print("✅ 분포 변화 탐지 라이브러리 로드 완료")


## 1. KS (Kolmogorov-Smirnov) 검정

수치형 데이터의 분포 변화를 탐지합니다.

In [ ]:
def detect_ks_drift(historical_data: np.ndarray, current_data: np.ndarray, threshold: float = 0.1) -> bool:
    """KS 검정을 사용하여 분포 변화를 감지"""
    ks_stat, p_value = stats.ks_2samp(historical_data, current_data)
    print(f"KS 통계량: {ks_stat:.4f}, p-값: {p_value:.4f}")
    return ks_stat > threshold

# 테스트: 쿼리 길이 분포 변화
historical_lengths = np.array([len(q) for q in ["What is the weather?", "Book a flight to Paris", "Recommend a book"] * 100])
current_lengths = np.array([len(q) for q in ["Query about latest AI news", "Longer user input with details"] * 150])

print("📊 KS 검정 결과:")
if detect_ks_drift(historical_lengths, current_lengths):
    print("⚠️ 편향 감지됨: 입력 변경 사항을 검토하세요.")
else:
    print("✅ 분포가 안정적입니다.")


## 2. PSI (Population Stability Index)

범주형 데이터의 분포 변화를 측정합니다.


In [ ]:
def calculate_psi(expected: np.ndarray, actual: np.ndarray) -> float:
    """범주형 데이터에 대한 PSI 계산"""
    expected_percents = expected / np.sum(expected)
    actual_percents = actual / np.sum(actual)
    # 0으로 나누기 방지
    expected_percents = np.clip(expected_percents, 1e-10, 1)
    actual_percents = np.clip(actual_percents, 1e-10, 1)
    psi_values = (actual_percents - expected_percents) * np.log(actual_percents / expected_percents)
    return np.sum(psi_values)

# 테스트: 도구 호출 분포 변화
historical_tools = np.array([50, 30, 20])  # 'refund', 'cancel', 'modify'
current_tools = np.array([40, 40, 20])

psi = calculate_psi(historical_tools, current_tools)
print(f"📊 PSI: {psi:.4f}")
if psi > 0.25:
    print("🚨 주요 편향 발생: 개입 필요!")
elif psi > 0.1:
    print("⚠️ 도구 사용에 경미한 편향 발생")
else:
    print("✅ 분포가 안정적입니다.")
